# **Board2Board Trainer**
**Approach**


1.   The training pipeline consists of two core components:
Piece Classification: A ResNet50 model (pretrained on ImageNet) is fine-tuned to classify chessboard squares into 13 categories. Input images (64×64 crops) are resized to 224×224 to match the ResNet50 architecture requirements. Dataset splitting is performed with splitfolders to create training and validation sets, and training is regularized using data augmentation, early stopping, and learning rate reduction.
2.   Threshold Prediction: Separate Linear Regression models are trained on hand-engineered image features to predict optimal thresholds for board detection (Hough Line threshold and Agglomerative Clustering threshold, both in primary and secondary processing stages). These thresholds are later integrated into the OpenCV-based board processor for robust grid extraction.







*Libraries that require installing*

In [ ]:
!pip install scikit-learn
!pip install split-folders

## ResNet50 Fine-Tuning
A transfer learning setup is used:
* The pretrained ResNet50 backbone is loaded with ImageNet weights, excluding its top classification layers.
* All layers are frozen initially, then the last 10 layers are unfrozen for fine-tuning.
* A custom head is attached: convolutional outputs are flattened, followed by a dense layer with 128 ReLU units, and a final softmax layer that outputs predictions for 13 classes (piece types + empty square).
* The model is compiled with the Adam optimizer and categorical cross-entropy loss.
* Training uses an 80/20 split with real-time data augmentation. Early stopping (based on validation accuracy) and ReduceLROnPlateau (on validation loss) ensure convergence without overfitting.

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np
import splitfolders
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Creating the Neural Network

# Loading the model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Transfer Learning and Fine Tuning the Model on the custom dataset
for layer in base_model.layers:
  layer.trainable = False
#base_model.summary()

for layer in base_model.layers[-10:]:
  layer.trainable = True

# Create the final model
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(13, activation='softmax')(x) #classify into 13 classes

output_layer = predictions

# Compiling the model
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


# Training the model

# split the dataset: 80% training data and 20% validation data
splitfolders.ratio("/content/drive/MyDrive/dataset2",output="output_dataset", seed=42, ratio=(0.8, 0.2))

train_dir = "output_dataset/train"
val_dir = "output_dataset/val"

# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2])

val_datagen = ImageDataGenerator()

# 224x224 images are most suitable with ResNet50 model
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

# Early Stopping: check the validation accuracy
# if validation accuracy doesn't improve after 4 epoches, stop
# and restore the weights calculated in the highest accuracy epoch
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=4,
    restore_best_weights=True
)

# Reduce Learning Rate where there is no improvement in validation loss
# fir 3 epoches
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# Start training
history = model.fit(train_gen, validation_data=val_gen, epochs=50, callbacks=[early_stop,reduce_lr])

# Metrics
loss, accuracy = model.evaluate(val_gen)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# Save model
model.save("board2board.h5")

**Performance Evaluation: Confusion Matrix**

To assess classification performance:
* A confusion matrix is computed by comparing predicted labels against true labels from the validation set.
* The results are visualized as a heatmap (using Seaborn) with class labels on both axes.
* A classification_report provides precision, recall, and F1 scores for each class. This combination allows both high-level accuracy monitoring and detailed insight into misclassifications between specific piece types.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Confusion Matrix to ensure model is well trained
class_names = list(train_gen.class_indices.keys())

y_true = []
y_pred = []

for i in range(len(val_gen)):
    x_batch, y_batch = val_gen[i]
    y_true_batch = np.argmax(y_batch, axis=1)
    y_pred_batch = np.argmax(model.predict(x_batch), axis=1)

    y_true.extend(y_true_batch)
    y_pred.extend(y_pred_batch)

    if i >= val_gen.__len__():
        break

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

print(classification_report(y_true, y_pred, target_names=val_gen.class_indices.keys()))


## Linear Regression for Threshold Prediction

While CNNs handle piece recognition, board detection relies on image-specific thresholds. To avoid manually tuning them, regression models are trained to predict thresholds directly from image features.

**Pipeline**
1.  Dataset Preparation: Each dataset (e.g., primary_hough_training.csv) contains numerical image descriptors (intensity, entropy, edge density, etc.) and the corresponding optimal threshold.
Train-Test Split: Data is divided into 80% training and 20% testing.
Feature Scaling: Standardization ensures features have zero mean and unit variance.
2.  Model Training: A LinearRegression model is fit on the scaled features.
3.  Evaluation: Predictions are compared against ground truth using MSE and R² metrics. Scatter plots visualize predicted vs. true thresholds.
4.  Pipeline Export: A scikit-learn pipeline (scaler + regression model) is serialized with Joblib for integration into the OpenCV board processor.

**Separate Models**
1.  *Primary Houghline Threshold* — predicts the line detection threshold for initial board extraction.
2.  *Primary Clustering Threshold* — predicts the distance parameter for clustering intersection points into grid corners.
3.  *Secondary Houghline Threshold* — refines thresholds after perspective correction.
4.  *Secondary Clustering Threshold* — adjusts clustering in the warped image stage.

In [ ]:
# Primary Houghline Threshold Linear Regression Pipeline

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

# 1. Load Dataset
df = pd.read_csv("/content/drive/MyDrive/Board2Board/primary_hough_training.csv")

# Separate features and target
X = df.drop("hough_threshold", axis=1).values
y = df["hough_threshold"].values.reshape(-1, 1)

print("Feature columns:", df.columns.tolist())

# ----------------------------------------
# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train Linear Regression Model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 5. Evaluate
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")
print("Learned Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# 6. Plot
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("True Hough Threshold")
plt.ylabel("Predicted Hough Threshold")
plt.title("Linear Regression: True vs Predicted")
plt.grid(True)
plt.show()

# 7. Save Model
pipeline = make_pipeline(scaler, model)
joblib.dump(pipeline, "primary_hough_pipeline.pkl")


In [ ]:
# Primary Clustering Threshold Linear Regression Pipeline

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

# 1. Load Dataset
df = pd.read_csv("/content/drive/MyDrive/Board2Board/primary_clustering_training.csv")
print(df.columns.tolist())

# Separate features and target
X = df.drop(" cluster_threshold", axis=1).values
y = df[" cluster_threshold"].values.reshape(-1, 1)

print("Feature columns:", df.columns.tolist())

# ----------------------------------------
# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train Linear Regression Model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 5. Evaluate
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")
print("Learned Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# 6. Plot
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("True Cluster Threshold")
plt.ylabel("Predicted Cluster Threshold")
plt.title("Linear Regression: True vs Predicted")
plt.grid(True)
plt.show()

# 7. Save Model
pipeline = make_pipeline(scaler, model)
joblib.dump(pipeline, "primary_cluster_pipeline.pkl")


In [ ]:
# Secondary Houghline Threshold Linear Regression Pipeline


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

# 1. Load Dataset
df = pd.read_csv("/content/drive/MyDrive/Board2Board/secondary_hough_training.txt")
print(df.columns.tolist())

# Separate features and target
X = df.drop("hough_threshold", axis=1).values
y = df["hough_threshold"].values.reshape(-1, 1)

print("Feature columns:", df.columns.tolist())

# ----------------------------------------
# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train Linear Regression Model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 5. Evaluate
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")
print("Learned Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# 6. Plot
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("True Houghline Threshold")
plt.ylabel("Predicted Houghline Threshold")
plt.title("Linear Regression: True vs Predicted")
plt.grid(True)
plt.show()

# 7. Save Model
pipeline = make_pipeline(scaler, model)
joblib.dump(pipeline, "secondary_hough_pipeline.pkl")


In [ ]:
# Secondary Clustering Threshold Linear Regression Pipeline


import pandas as pd

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

# 1. Load Dataset
df = pd.read_csv("/content/drive/MyDrive/Board2Board/secondary_clustering_training.txt")
print(df.columns.tolist())

# Separate features and target
X = df.drop(" cluster_threshold", axis=1).values
y = df[" cluster_threshold"].values.reshape(-1, 1)

print("Feature columns:", df.columns.tolist())

# ----------------------------------------
# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Standardize Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Train Linear Regression Model
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# 5. Evaluate
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")
print("Learned Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

# 6. Plot
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("True Cluster Threshold")
plt.ylabel("Predicted Cluster Threshold")
plt.title("Linear Regression: True vs Predicted")
plt.grid(True)
plt.show()

# 7. Save Model
pipeline = make_pipeline(scaler, model)
joblib.dump(pipeline, "secondary_cluster_pipeline.pkl")


**© 2025 Jared Aung. All rights reserved.**
